##### ----- TFIDF

這部分與'kaggle_1.ipynb'的差別在於使用TF-IDF將文本向量化，而不是採用BOW，

那兩者最大的差別就是有無考慮到詞彙本身的重要性，

TF-IDF中的IDF的就是詞彙出現在所有文本中的次數之倒數，

意思是說當這個詞彙出現在越多文本中則代表其重要性越低(所以才要取倒數)。

In [4]:
import pandas as pd
import numpy as np

In [5]:
## load a pickle file
train_df = pd.read_pickle("train_df.pkl")

train_df.drop('identification', axis=1, inplace=True)
train_df

,tweet_id,text,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation
...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,joy
1455560,0x2cbca6,there's currently two girls walking around the...,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",joy


In [6]:
test_df = pd.read_pickle("test_df.pkl")

test_df.drop('identification', axis=1, inplace=True)
test_df

,tweet_id,text
2,0x28b412,"Confident of your obedience, I write to you, k..."
4,0x2de201,"""Trust is not the same as faith. A friend is s..."
9,0x218443,When do you have enough ? When are you satisfi...
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #..."
33,0x26289a,"In these tough times, who do YOU turn to as yo..."
...,...,...
1867525,0x2913b4,"""For this is the message that ye heard from th..."
1867529,0x2a980e,"""There is a lad here, which hath five barley l..."
1867530,0x316b80,When you buy the last 2 tickets remaining for ...
1867531,0x29d0cb,I swear all this hard work gone pay off one da...


In [7]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# nltk.download('punkt')

## build analyzers (bag-of-words)
## Tokenizer
TFIDF = TfidfVectorizer(max_features=30000, tokenizer=nltk.word_tokenize) 

## apply analyzer to training data
TFIDF.fit(train_df['text'])

# train_data_TFIDF_features = TFIDF.transform(train_df['text'])

## check dimension
# train_data_TFIDF_features.shape

c:\Users\Martin\Desktop\Master_1\Virtualenv\Python311_ML\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_features=30000,
                tokenizer=<function word_tokenize at 0x000002A89D234180>)

In [8]:
# print(pd.DataFrame(train_data_BOW_features.toarray(), columns = BOW.get_feature_names_out()))

# # Show feature names for features [100:110]
# feature_names = BOW.get_feature_names_out()
# print("Feature names for features [100:110]:", feature_names[100:110])

In [9]:
# ---training data (features)
X_train = TFIDF.transform(train_df['text'])
# ---training labels
y_train = train_df['emotion']

In [10]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import keras

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)

In [12]:
from sklearn.neural_network import MLPRegressor

# Create an MLP regression model with verbose training
MLP_regressor = MLPRegressor(activation='logistic',
                             solver='adam',
                             alpha=0.00007,
                             max_iter=10,
                             hidden_layer_sizes=(32,32,32),
                             verbose=True,
                             early_stopping=False)

# Training the MLP regression model
MLP_regressor.fit(X_train, y_train)

Iteration 1, loss = 0.04393775
Iteration 2, loss = 0.04140557
Iteration 3, loss = 0.04096187
Iteration 4, loss = 0.04085558
Iteration 5, loss = 0.04058224
Iteration 6, loss = 0.04037998
Iteration 7, loss = 0.04025178
Iteration 8, loss = 0.04012721
Iteration 9, loss = 0.04000673
Iteration 10, loss = 0.03990311


c:\Users\Martin\Desktop\Master_1\Virtualenv\Python311_ML\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(activation='logistic', alpha=7e-05,
             hidden_layer_sizes=(32, 32, 32), max_iter=10, verbose=True)

In [13]:
# Predictions on training and testing data
y_train_pred_MLP = MLP_regressor.predict(X_train)
print(y_train_pred_MLP[0:2])

[[ 0.0726131   0.07843914  0.3385758   0.02931836  0.0956221   0.14443644
   0.12899079  0.10956952]
 [-0.00758652  0.02303444  0.23917265  0.00157409 -0.01774891  0.63391591
   0.07770075  0.04962933]]


In [14]:
pred_train_result = label_decode(label_encoder, y_train_pred_MLP)
pred_train_result[:2]

array(['disgust', 'sadness'], dtype=object)

In [15]:
# ---training data (features)
X_test = TFIDF.transform(test_df['text'])
y_test_pred_MLP = MLP_regressor.predict(X_test)
pred_test_result = label_decode(label_encoder, y_test_pred_MLP)
pred_test_result[:2]

array(['anticipation', 'anticipation'], dtype=object)

In [16]:
submission_df = pd.DataFrame({
    'id':test_df['tweet_id'],
    'emotion':pred_test_result,
})

submission_df

,id,emotion
2,0x28b412,anticipation
4,0x2de201,anticipation
9,0x218443,joy
30,0x2939d5,joy
33,0x26289a,trust
...,...,...
1867525,0x2913b4,anticipation
1867529,0x2a980e,anticipation
1867530,0x316b80,sadness
1867531,0x29d0cb,anger


In [ ]:
submission_df.to_csv('submission_3.csv', index=False)